In [1]:
using Dates
using PRMaps
using Plots
using Healpix

using thesis

import Pandas as Pd
import Stripeline as Sl

In [2]:
function pipeline_save(error)
        
    t_start = DateTime(2022, 1, 1, 12, 0, 0)
    sky_model = "s0"
    nside = 512
    obs_days = 10
     
    setup = PRMaps.Setup(
        sampling_freq_Hz = 50.,
        total_time_s = 24. * 3600. * obs_days
    )

    strip = Pd.query(Pd.read_pickle("../instruments/lspe_strip_instrument.pkl"), :(frequency==43.0))
    signal = get_foreground_maps(strip, sky_model, nside)[1]

    # We simulate the I0 horn that point towards [0.0,0.0,1.0]
    cam_ang = Sl.CameraAngles()
    
    ideal_map, _ = makeIdealMapIQU(cam_ang, signal, setup, t_start)
    
    w2_TA = Sl.TelescopeAngles(wheel2ang_0_rad = error)
    w3_TA = Sl.TelescopeAngles(wheel3ang_0_rad = error)
    fork_TA = Sl.TelescopeAngles(forkang_rad = error)
    wω_TA_0 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(0.0))
    wω_TA_90 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(90.0))
    wω_TA_180 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(180.0))
    wω_TA_270 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(270.0))


    w2_Map, w2_hit = makeErroredMapIQU(cam_ang, w2_TA, signal, setup, t_start)
    w3_Map, w3_hit = makeErroredMapIQU(cam_ang, w3_TA, signal, setup, t_start)
    fork_Map, fork_hit = makeErroredMapIQU(cam_ang, fork_TA, signal, setup, t_start)
    wω0_Map, wω0_hit = makeErroredMapIQU(cam_ang, wω_TA_0, signal, setup, t_start)
    wω90_Map, wω90_hit = makeErroredMapIQU(cam_ang, wω_TA_90, signal, setup, t_start)
    wω180_Map, wω180_hit = makeErroredMapIQU(cam_ang, wω_TA_180, signal, setup, t_start)
    wω270_Map, wω270_hit = makeErroredMapIQU(cam_ang, wω_TA_270, signal, setup, t_start)

    errored_maps = [w2_Map, w3_Map, fork_Map, wω0_Map, wω90_Map, wω180_Map, wω270_Map]
    errored_hit = [w2_hit, w3_hit, fork_hit, wω0_hit, wω90_hit, wω180_hit, wω270_hit]
    return (errored_maps, errored_hit, ideal_map)
end

function save(error)
    (errored, hit, ideal) = pipeline_save(error)
    saveToFITS(ideal, "!maps/maps/ideal.fits")
    labels = ["altitude_offset" "ground_offset" "fork" "zVAX&ω_0" "zVAX&ω_90" "zVAX&ω_180" "zVAX&ω_270"]
    for (i, map) in enumerate(errored)
        name = "!maps/maps/"*labels[i]*".fits"
        name_hit = "!maps/hit_maps/"*labels[i]*"_hit.fits"
        saveToFITS(map, name)
        saveToFITS(hit[i], name_hit)
    end
end

save (generic function with 1 method)

In [3]:
save(deg2rad(1.0/60))